# Imports

In [ ]:
# system and data handling packages
import os
import sys
import json
import csv
import collections as col

# matplotlib packages
%matplotlib inline
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
import matplotlib.pyplot as plt
# change figure fonts
from matplotlib import rcParams

# import jsci, CT's enhanced json stream write package
import jsci
from jsci import WriteStream as jsciwrite
from jsci import Coding as jscicoding

# numpy
import numpy as np
from scipy.interpolate import interp1d
import scipy as sp
from scipy import stats
from scipy import optimize as scop
import scipy.integrate as spint
import scipy.fftpack as spft
import scipy.signal as spsig

# import Kitaev Honeycomb package
import kithcmb
from kithcmb import ThermalGradient as vs

In [ ]:
plt.style.use('prettyfigs')

In [ ]:
# These are the "Tableau 20" colors as RGB.       
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),  
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),  
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),  
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]
# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.)  

# Read data

In [ ]:
data_paths = []
data_paths.append( os.pardir + '/data/2017-04-07_1312_rIt' )

In [ ]:
L_set = np.array([6])
max_t = '3000.0'
dt = '0.1'
T_set = np.arange(1,11)
dpsi_set = np.arange(1,11)
J = '1.0'
K = '0.1'
samples = 100
#
temps = np.array([ 10**( -2 + 2./9*(T-1) ) for T in T_set ])
dpsis = np.array([ 10**( -2 + 2./9*(dp-1) ) for dp in dpsi_set ])
times = np.arange( 0.,float(max_t),float(dt) )

In [ ]:
def read_data( L_,T_,dpsi_,sample_,search_dirs=data_paths ):
    for dpath in search_dirs:
        try:
            filename = 'response-current_NV_L'+str(L_)+'_T'+str(T_)+'_tmax'+max_t+'_dt'+dt+'_J'+J+'_K'+K+'_dpsi'+str(dpsi_)+'_sample'+str(sample_)+'.json'
            with open( dpath+'/'+filename, 'r') as file:
                data = json.load( file, cls=jscicoding.NumericDecoder )
            return data
        except IOError:
            pass
        except ValueError:
            # print 'corrupt data file ',dpath+'/'+filename
            pass
    raise IOError

In [ ]:
def load_data( L_,T_,dpsi_ ):
    print 'L : ',L_,', T : ',T_,', dpsi : ',dpsi_
    # read the first sample as the base data file
    for sample in range(1,samples+1):
        try:
            data = read_data(L_,T_,dpsi_,sample)
            break
        except IOError:
            if sample==samples:
                print 'found no files for ',(L,T)
                raise IOError
            pass
    
    # rearrange the data to have the time series in an easier format
    data['xcurr'] = np.zeros(len(data['time_series']))
    data['zcurr'] = np.zeros(len(data['time_series']))
    dpsi = float(dpsi_)
    
    # iterate over the rest of the samples, and combine time-series data
    samples_found = 0
    for directory in [ '2017-04-07_1312_rIt' ]:
        for sample in range(1,samples+1):
            try:
                temp = read_data(L_,T_,dpsi_,sample,search_dirs=[ os.pardir+'/data/'+directory ])
                tempx = []
                tempz = []
                for t_series in temp['time_series']:
                    tempx.append(t_series['x']*(1./(2*L_)))
                    tempz.append(t_series['z']*(1./(2*L_)))
                data['xcurr'] = data['xcurr'] + np.array(tempx)
                data['zcurr'] = data['zcurr'] + np.array(tempz)
                samples_found = samples_found+1
            except IOError:
                #print 'could not find file for ',(L,T,sample)
                pass
            
    # normalise by number of samples
    print 'found sample : ',samples_found
    data['samples_found'] = samples_found
    data['xcurr'] = 1./samples_found * data['xcurr']
    data['zcurr'] = 1./samples_found * data['zcurr']
        
    # correct x-component
    data['xcurr'] = 2./np.sqrt(3) * (np.array(data['xcurr']) + 0.5*np.array(data['zcurr']))
        
    return data

In [ ]:
data = {}
for L in L_set:
    for T in T_set:
        for dpsi in dpsi_set:
            try:
                data[L,T,dpsi] = load_data(L,T,dpsi)
            except IOError:
                pass

# Analyse

## Plot the time-series

In [ ]:
L = 6
T = 8
print 'T : ',temps[T]
for dpsi in dpsi_set:
    print 'dpsi : ',dpsi
    
    fig,ax = plt.subplots()
    plt.plot( times,data[L,T,dpsi]['xcurr'],'.-' )
    plt.plot( times,data[L,T,dpsi]['zcurr'],'.-' )
    #plt.plot( times[1:],spint.cumtrapz(data[L,T,dpsi]['zcurr'],dx=float(dt)),'.-' )
    plt.xlabel(r'$t$')
    plt.ylabel(r'$I$')
    #ax.set_xlim([0,5])

    #plt.savefig( 'quench-response-currents-lowT.pdf', bbox_inches='tight', transparency=True )
    plt.show()

# Investigate s->0 limit

We compute the Laplace transform of the signal in the s->0 limit, where the limit is taken along on the real axis.

In [ ]:
def lin_fit(x,a,b):
    return a*x + b

In [ ]:
s_vals = 1./np.linspace(100,600,20)
for L in L_set:
    for dpsi in dpsi_set:
        
        print 'L : ',L,' dpsi : ',dpsi
        fig,(ax1,ax2) = plt.subplots(1,2)
        for T in T_set:

            try:
                x_series = []
                y_series = []
                for s in s_vals:
                    x_series.append(spint.simps(data[L,T,dpsi]['xcurr']*np.exp(-s*times),dx=float(dt)))
                    y_series.append(spint.simps(data[L,T,dpsi]['zcurr']*np.exp(-s*times),dx=float(dt)))
                data_line1, = ax1.plot( 1./s_vals,x_series,'o',label=r'$T=$'+"{0:.2g}".format(temps[T-1]) )
                data_line2, = ax2.plot( 1./s_vals,y_series,'o',label=r'$T=$'+"{0:.2g}".format(temps[T-1]) )
                
                # try to fit
                popt, pcov = scop.curve_fit(lin_fit, 1./s_vals,x_series)
                xfit=np.linspace(1./s_vals[-1],1./s_vals[0],60)
                ax1.plot( xfit,lin_fit(xfit,*popt),color=data_line1.get_color() )
                data[L,T,dpsi]['Dx'] = popt[0]
                data[L,T,dpsi]['Cx'] = popt[1]
                #
                popt, pcov = scop.curve_fit(lin_fit, 1./s_vals,y_series)
                xfit=np.linspace(1./s_vals[-1],1./s_vals[0],60)
                ax2.plot( xfit,lin_fit(xfit,*popt),color=data_line2.get_color() )
                data[L,T,dpsi]['Dy'] = popt[0]
                data[L,T,dpsi]['Cy'] = popt[1]
                
            except KeyError:
                pass

        ax1.set_title('$x$-dirn')
        ax1.set_xlabel(r'$1/s$')
        ax1.legend(loc=3)
        ax2.plot( [],[] )
        ax2.set_title('$y$-dirn')
        ax2.set_xlabel(r'$1/s$')
        #ax2.legend(loc=3)
        plt.show()
        

In [ ]:
s_vals = 1./np.linspace(100,600,20)
for L in L_set:
    for T in T_set:
        print 'L : ',L,' T : ',temps[T-1]
        fig,(ax1,ax2) = plt.subplots(1,2)
        x_series = []
        y_series = []
        dpsi_vals = []
        for dpsi in dpsi_set:
            try:
                x_series.append(data[L,T,dpsi]['Dx'])
                y_series.append(data[L,T,dpsi]['Dy'])
                dpsi_vals.append(dpsis[dpsi-1])
            except KeyError:
                pass

        ax1.plot( dpsi_vals,x_series,'o-' )
        ax2.plot( dpsi_vals,y_series,'o-' )

        ax1.set_title('$x$-dirn')
        ax1.set_xlabel(r'$\Delta \phi$')
        ax1.set_ylabel(r'$D_x$')
        ax1.legend(loc=3)
        ax2.plot( [],[] )
        ax2.set_title('$y$-dirn')
        ax2.set_xlabel(r'$\Delta \phi$')
        ax2.set_ylabel(r'$D_y$')
        #ax2.legend(loc=3)
        fig.tight_layout()
        plt.show()


In [ ]:
s_vals = 1./np.linspace(100,600,20)
for L in L_set:
    for T in T_set:
        print 'L : ',L,' T : ',temps[T-1]
        fig,(ax1,ax2) = plt.subplots(1,2)
        x_series = []
        y_series = []
        dpsi_vals = []
        for dpsi in dpsi_set:
            try:
                x_series.append(data[L,T,dpsi]['Cx'])
                y_series.append(data[L,T,dpsi]['Cy'])
                dpsi_vals.append(dpsis[dpsi-1])
            except KeyError:
                pass

        ax1.plot([],[])
        ax1.plot( dpsi_vals,x_series,'o-' )
        ax2.plot([],[])
        ax2.plot( dpsi_vals,y_series,'o-' )

        ax1.set_title('$x$-dirn')
        ax1.set_xlabel(r'$\Delta \phi$')
        ax1.set_ylabel(r'$C_x$')
        ax1.legend(loc=3)
        ax2.plot( [],[] )
        ax2.set_title('$y$-dirn')
        ax2.set_xlabel(r'$\Delta \phi$')
        ax2.set_ylabel(r'$C_y$')
        #ax2.legend(loc=3)
        fig.tight_layout()
        plt.show()


In [ ]:
for L in L_set:
    print 'L : ',L
    fig,ax = plt.subplots()
    for T in T_set[::3]:
        dataline, = plt.plot( data[L,T]['freq'],data[L,T]['zpower'].real,',')
        plt.plot( data[L,T]['freq'],data[L,T]['smooth_zpower'].real,'-',label=r'$T=$'+"{0:.2g}".format(temps[T-T_set[0]]),color=dataline.get_color() )
    plt.xlabel(r'$\nu$')
    plt.ylabel(r'$| \, H( \, ^1 \! /_{200} + 2\pi i\nu \, ) \, |$')
    plt.xlim([-0.01,1.])
    #plt.ylim([-10,10])
    plt.legend(loc=4)
    #ax.set_yscale('log')

    #plt.savefig( 'long_curr-curr-'+correl+'.pdf',  bbox_extra_artist=(leg,), bbox_inches='tight', transparency=True )
    plt.show()

In [ ]:
for L in L_set:
    print 'L : ',L
    fig,ax = plt.subplots()
    for T in T_set[::3]:
        dataline, = plt.plot( data[L,T]['freq'],data[L,T]['xpower'].real,',')
        plt.plot( data[L,T]['freq'],data[L,T]['smooth_xpower'].real,'-',label=r'$T=$'+"{0:.2g}".format(temps[T-T_set[0]]),color=dataline.get_color() )
    plt.xlabel(r'$\nu$')
    plt.ylabel(r'$| \, H( \, ^1 \! /_{200} + 2\pi i\nu \, ) \, |$')
    plt.xlim([-0.01,0.5])
    #plt.ylim([-10,10])
    plt.legend(loc=4)
    #ax.set_yscale('log')

    #plt.savefig( 'long_curr-curr-'+correl+'.pdf',  bbox_extra_artist=(leg,), bbox_inches='tight', transparency=True )
    plt.show()

Output the I(L,T,\omega=0) values

In [ ]:
fig,ax = plt.subplots()
runtimes = []
for L in L_set:
    max_runtime = 0.
    for T in T_set:
        max_runtime = max([ data[L,T]['run_time'], max_runtime ])
    runtimes.append(max_runtime / 3600.)
plt.plot( L_set,runtimes,'o-')

plt.xlabel(r'$L$')
plt.ylabel('run time / hours')
plt.legend(loc=4,fontsize=16)
#ax.set_yscale('log')

#plt.savefig( 'long_curr-curr-'+correl+'.pdf',  bbox_extra_artist=(leg,), bbox_inches='tight', transparency=True )
plt.show()

In [ ]:
L = 16
max_runtime = 0.
for T in T_set:
    max_runtime = max([ data[L,T]['run_time'], max_runtime ])
print max_runtime/3600.

In [ ]:
fig,ax = plt.subplots()
for L in L_set:
    L_series = []
    for T in T_set:
        L_series.append( np.real(data[L,T]['zspectrum'][0]) )
    plt.plot( temps,L_series,'o-',label=r'$L=$'+str(L) )

plt.plot( temps,[0. for x in temps], 'k--' )
plt.xlabel(r'$T$')
#plt.ylabel(r'$L \,\cdot \, \mathcal{F} \, [I_x]( \omega = 0 )$')
plt.legend(loc=4,fontsize=16)
#plt.xlim([0,0.5])
#plt.ylim([0,200])
ax.semilogx()

#plt.savefig( 'response-current-X.pdf', bbox_inches='tight', transparency=True )
plt.show()

In [ ]:
fig,ax = plt.subplots()
for L in L_set:
    L_series = []
    for T in T_set:
        L_series.append( np.real(data[L,T]['xpower'][0]) )
    plt.plot( temps,L_series,'o-',label=r'$L=$'+str(L) )

plt.plot( temps,[0. for x in temps], 'k--' )
plt.xlabel(r'$T$')
#plt.ylabel(r'$L \,\cdot \, \mathcal{F} \, [I_x]( \omega = 0 )$')
plt.legend(loc=4,fontsize=16)
#plt.xlim([0,0.5])
plt.ylim([-0.1,0.1])
ax.semilogx()

#plt.savefig( 'response-current-X.pdf', bbox_inches='tight', transparency=True )
plt.show()

In [ ]:
fig,ax = plt.subplots()
for L in L_set:
    L_series = []
    for T in T_set:
        L_series.append( np.real(data[L,T]['xpower'][0]/(1-data[L,T]['zspectrum'][0])) )
    plt.plot( temps,np.array(L_series),'o-',label=r'$L=$'+str(L) )

plt.plot( temps,[0. for x in temps], 'k--' )
plt.xlabel(r'$T$')
#plt.ylabel(r'$L \,\cdot \, \mathcal{F} \, [I_x]( \omega = 0 )$')
plt.legend(loc=4,fontsize=16)
#plt.xlim([0,0.5])
#plt.ylim([-0.1,0.1])

#plt.savefig( 'response-current-X.pdf', bbox_inches='tight', transparency=True )
plt.show()

In [ ]:
fig,ax = plt.subplots()
for L in L_set:
    L_series = []
    for T in T_set:
        L_series.append( data[L,T]['zintegral'] )
    plt.plot( temps,L_series,'o-',label=r'$L=$'+str(L) )

plt.xlabel(r'$T$')
plt.ylabel(r'$L \,\cdot \, \mathcal{F} \, [I_z](\omega = 0 )$')
plt.legend(loc=4,fontsize=16)
#plt.xlim([0,0.2])
#plt.ylim([0,0.03])

#plt.savefig( 'response-current-Z.pdf', bbox_inches='tight', transparency=True )
plt.show()